In [ ]:
%create_livy_session \
--cluster ephemeral-cluster \
--id ses1 \
--conf spark.jars.packages=io.delta:delta-core_2.12:0.8.0 \
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension \
--conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog \
--conf spark.sql.hive.metastore.sharedPrefixes=com.amazonaws,ru.yandex.cloud \
--conf spark.sql.warehouse.dir=s3a://keshaaa/wh

Waiting for Apache Livy to start...
Apache Livy has started.
Starting session ses1


In [ ]:
#!spark --cluster ephemeral-cluster --session ses1
S3_BUCKET_NAME="keshaaa"
inputDF = spark.createDataFrame(
    [
        ("100", "2015-01-01", "2015-01-01T13:51:39.340396Z"),
        ("101", "2015-01-01", "2015-01-01T12:14:58.597216Z"),
        ("102", "2015-01-01", "2015-01-01T13:51:40.417052Z"),
        ("103", "2015-01-01", "2015-01-01T13:51:40.519832Z"),
        ("104", "2015-01-02", "2015-01-01T12:15:00.512679Z"),
        ("105", "2015-01-02", "2015-01-01T13:51:42.248818Z"),
    ],
    ["id", "creation_date", "last_update_time"],
)

# Write a DataFrame as a Delta dataset
inputDF.write.format("delta").mode("overwrite").option(
    "overwriteSchema", "true"
).partitionBy("creation_date").save(f"s3a://{S3_BUCKET_NAME}/tmp/delta/")

Waiting for an Apache Livy session to start...
Apache Livy session has started.



In [11]:
%delete_livy_session \
--cluster ephemeral-cluster \
--id ses1

In [ ]:
# %pip install boto3 if needed
# set os environment variables aws_access_key_id and aws_secret_access_key
import os
import boto3

S3_CREDS = {
    "aws_access_key_id": os.environ['aws_access_key_id'],
    "aws_secret_access_key": os.environ['aws_secret_access_key']
}
bucket_name = "my_bucket"

source_path = 'path/to/remote/directory'
target_path = 'path/to/local/directory'

s3r = boto3.resource(service_name='s3', endpoint_url='https://storage.yandexcloud.net', **S3_CREDS)
bucket = s3r.Bucket(bucket_name)

for obj in bucket.objects.filter(Prefix=source_path):
    if not os.path.relpath(obj.key, source_path).startswith('../'):
        os.makedirs(os.path.join(target_path, os.path.dirname(obj.key)), exist_ok=True)
        if obj.key[-1] != '/':
            bucket.download_file(obj.key, os.path.join(target_path, obj.key))
